In [ ]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()

In [ ]:
using LinearAlgebra
using PyPlot
using ForwardDiff
using RobotZoo
using RobotDynamics
using MatrixCalculus
using JLD2

In [ ]:
#Acrobot Dynamics
a = RobotZoo.Acrobot()
h = 0.05

In [ ]:
function dynamics_rk4(x,u)
    #RK4 integration with zero-order hold on u
    f1 = RobotZoo.dynamics(a, x, u)
    f2 = RobotZoo.dynamics(a, x + 0.5*h*f1, u)
    f3 = RobotZoo.dynamics(a, x + 0.5*h*f2, u)
    f4 = RobotZoo.dynamics(a, x + h*f3, u)
    return x + (h/6.0)*(f1 + 2*f2 + 2*f3 + f4)
end

In [ ]:
function dfdx(x,u)
    ForwardDiff.jacobian(dx->dynamics_rk4(dx,u),x)
end

function dfdu(x,u)
    ForwardDiff.derivative(du->dynamics_rk4(x,du),u)
end

In [ ]:
function dAdx(x,u)
    ForwardDiff.jacobian(dx->vec(dfdx(dx,u)),x)
end

function dBdx(x,u)
    ForwardDiff.jacobian(dx->dfdu(dx,u),x)
end

function dAdu(x,u)
    ForwardDiff.derivative(du->vec(dfdx(x,du)),u)
end

function dBdu(x,u)
    ForwardDiff.derivative(du->dfdu(x,du),u)
end

In [ ]:
Nx = 4     # number of state
Nu = 1     # number of controls
Tfinal = 10.0 # final time
Nt = Int(Tfinal/h)+1    # number of time steps
thist = Array(range(0,h*(Nt-1), step=h));

In [ ]:
# Cost weights
Q = Diagonal([1.0*ones(2); 0.1*ones(2)]);
R = 0.01;
Qn = Array(100.0*I(Nx));

In [ ]:
function stage_cost(x,u)
    return 0.5*((x-xgoal)'*Q*(x-xgoal)) + 0.5*R*u*u
end

In [ ]:
function terminal_cost(x)
    return 0.5*(x-xgoal)'*Qn*(x-xgoal)
end

In [ ]:
function cost(xtraj,utraj)
    J = 0.0
    for k = 1:(Nt-1)
        J += stage_cost(xtraj[:,k],utraj[k])
    end
    J += terminal_cost(xtraj[:,Nt])
    return J
end

In [ ]:
function backward_pass!(p,P,d,K)
    
    ΔJ = 0.0
    p[:,Nt] .= Qn*(xtraj[:,Nt]-xgoal)
    P[:,:,Nt] .= Qn
    
    for k = (Nt-1):-1:1
        #Calculate derivatives
        q = Q*(xtraj[:,k]-xgoal)
        r = R*utraj[k]
    
        A = dfdx(xtraj[:,k], utraj[k])
        B = dfdu(xtraj[:,k], utraj[k])
    
        gx = q + A'*p[:,k+1]
        gu = r + B'*p[:,k+1]
    
        #iLQR (Gauss-Newton) version
        Gxx = Q + A'*P[:,:,k+1]*A
        Guu = R + B'*P[:,:,k+1]*B
        Gxu = A'*P[:,:,k+1]*B
        Gux = B'*P[:,:,k+1]*A
        
        #DDP (full Newton) version
        #Ax = dAdx(xtraj[:,k], utraj[k])
        #Bx = dBdx(xtraj[:,k], utraj[k])
        #Au = dAdu(xtraj[:,k], utraj[k])
        #Bu = dBdu(xtraj[:,k], utraj[k])
        #Gxx = Q + A'*P[:,:,k+1]*A + kron(p[:,k+1]',I(Nx))*comm(Nx,Nx)*Ax
        #Guu = R + B'*P[:,:,k+1]*B + (kron(p[:,k+1]',I(Nu))*comm(Nx,Nu)*Bu)[1]
        #Gxu = A'*P[:,:,k+1]*B + kron(p[:,k+1]',I(Nx))*comm(Nx,Nx)*Au
        #Gux = B'*P[:,:,k+1]*A + kron(p[:,k+1]',I(Nu))*comm(Nx,Nu)*Bx
        
        #β = 0.1
        #while !isposdef(Symmetric([Gxx Gxu; Gux Guu]))
        #    Gxx += β*I
        #    Guu += β*I
        #    β = 2*β
        #    display("regularizing G")
        #    display(β)
        #end
        
        d[k] = Guu\gu
        K[:,:,k] .= Guu\Gux
    
        #p[:,k] .= dropdims(gx - K[:,:,k]'*gu + K[:,:,k]'*Guu*d[k] - Gxu*d[k], dims=2)
        p[:,k] = gx - K[:,:,k]'*gu + K[:,:,k]'*Guu*d[k] - Gxu*d[k]
        P[:,:,k] .= Gxx + K[:,:,k]'*Guu*K[:,:,k] - Gxu*K[:,:,k] - K[:,:,k]'*Gux
    
        ΔJ += gu'*d[k]
    end
    
    return ΔJ
end

In [ ]:
#Initial guess
x0 = [-pi/2; 0; 0; 0]
xgoal = [pi/2; 0; 0; 0]
xtraj = kron(ones(1,Nt), x0)
utraj = randn(Nt-1);
#f = jldopen("guess.jld2", "r")
#utraj = f["utraj"];

In [ ]:
#Initial Rollout
for k = 1:(Nt-1)
    xtraj[:,k+1] .= dynamics_rk4(xtraj[:,k],utraj[k])
end
J = cost(xtraj,utraj)

In [ ]:
#DDP Algorithm
using Printf
p = ones(Nx,Nt)
P = zeros(Nx,Nx,Nt)
d = ones(Nt-1)
K = zeros(Nu,Nx,Nt-1)
ΔJ = 0.0

xn = zeros(Nx,Nt)
un = zeros(Nt-1)

gx = zeros(Nx)
gu = 0.0
Gxx = zeros(Nx,Nx)
Guu = 0.0
Gxu = zeros(Nx)
Gux = zeros(Nx)

iter = 0
while maximum(abs.(d[:])) >  1e-3
    iter += 1    
    
    #Backward Pass
    ΔJ = backward_pass!(p,P,d,K)

    #Forward rollout with line search
    xn[:,1] = xtraj[:,1]
    α = 1.0
    for k = 1:(Nt-1)
        un[k] = utraj[k] - α*d[k] - dot(K[:,:,k],xn[:,k]-xtraj[:,k])
        xn[:,k+1] .= dynamics_rk4(xn[:,k],un[k])
    end
    Jn = cost(xn,un)
    
    while isnan(Jn) || Jn > (J - 1e-2*α*ΔJ)
        α = 0.5*α
        for k = 1:(Nt-1)
            un[k] = utraj[k] - α*d[k] - dot(K[:,:,k],xn[:,k]-xtraj[:,k])
            xn[:,k+1] .= dynamics_rk4(xn[:,k],un[k])
        end
        Jn = cost(xn,un)
    end

    # logging
    if rem(iter - 1, 100) == 0
        @printf "iter     J           ΔJ        |d|         α       \n"
        @printf "---------------------------------------------------\n"
    end
    if rem(iter - 1, 10) == 0 
        @printf("%3d   %10.3e  %9.2e  %9.2e  %6.4f  \n",
              iter, J, ΔJ, maximum(abs.(d[:])), α)
    end
    
    J = Jn
    xtraj .= xn
    utraj .= un
end

In [ ]:
plot(thist,xtraj[1,:])
plot(thist,xtraj[2,:])

In [ ]:
plot(thist[1:Nt-1],utraj)

In [ ]:
# Acrobot (doublependulum)
import MeshCat as mc
using Colors

function RotX(alpha)
    c, s = cos(alpha), sin(alpha)
    [1 0 0; 0 c -s; 0 s  c]
end

function create_acrobot!(vis, color=colorant"blue", thick=0.05)
    l1,l2 = [1.,1.]
    hinge = mc.Cylinder(mc.Point(-0.05,0,0), mc.Point(0.05,0,0), 0.05)
    dim1  = mc.Vec(thick, thick, l1)
    link1 = mc.HyperRectangle(mc.Vec(-thick/2,-thick/2,0),dim1)
    dim2  = mc.Vec(thick, thick, l2)
    link2 = mc.HyperRectangle(mc.Vec(-thick/2,-thick/2,0),dim2)
    mat1 = mc.MeshPhongMaterial(color=colorant"grey")
    mat2 = mc.MeshPhongMaterial(color=color)
    mc.setobject!(vis["base"], hinge, mat1) 
    mc.setobject!(vis["link1"], link1, mat2) 
    mc.setobject!(vis["link1","joint"], hinge, mat1) 
    mc.setobject!(vis["link1","link2"], link2, mat2) 
    mc.settransform!(vis["link1","link2"], mc.Translation(0,0,l1))
    mc.settransform!(vis["link1","joint"], mc.Translation(0,0,l1))
end

function update_acro_pose!(vis, x)
    l1, l2 = [1, 1.]
    mc.settransform!(vis["robot","link1"], mc.LinearMap(RotX(x[1]-pi/2)))
    mc.settransform!(vis["robot","link1","link2"], mc.compose(mc.Translation(0,0,l1), mc.LinearMap(RotX(x[2]))))
end

function animate_acrobot(X, dt)
    vis = mc.Visualizer()
    create_acrobot!(vis["robot"])
    anim = mc.Animation(vis; fps=floor(Int,1/dt))
    for k = 1:length(X)
        mc.atframe(anim, k) do
            update_acro_pose!(vis,X[k])
        end
    end
    mc.setanimation!(vis, anim)
    return mc.render(vis)
end

In [ ]:
X1 = [Vector(x) for x in eachcol(xtraj)];
animate_acrobot(X1, h)